In [1]:
!ls '../'

input  lib  working


In [2]:
!ls '../input/'

coco-2017-dataset	     thesis-imgcap-run2-deterministic-info
imgcap-kagg-run2-weights-in


In [3]:
!ls '../input/thesis-imgcap-run2-deterministic-info/'

descriptions_test.pkl		       img_encodings_train_97000.pkl
descriptions_train_97000.pkl	       img_encodings_val_3000.pkl
descriptions_train_97000_startend.pkl  ixtoword_train_97000.pkl
descriptions_train_and_val.pkl	       train_imgs_keys_97000_randomSplit444.pkl
descriptions_val_3000.pkl	       val_imgs_keys_3000_randomSplit444.pkl
embedding_matrix_6758_200.pkl	       wordtoix_train_97000.pkl


In [4]:
!ls '../input/imgcap-kagg-run2-weights-in/'

Decoder_Run_2_Wt_ep_1.h5  Decoder_Run_2_Wt_ep_4.h5  Decoder_Run_2_Wt_ep_7.h5
Decoder_Run_2_Wt_ep_2.h5  Decoder_Run_2_Wt_ep_5.h5
Decoder_Run_2_Wt_ep_3.h5  Decoder_Run_2_Wt_ep_6.h5


In [5]:
!ls '../working/'

__notebook_source__.ipynb


In [6]:
!mkdir '../working/weights_out/'

In [7]:
!ls '../working/'

__notebook_source__.ipynb  weights_out


In [8]:
!ls '../working/weights_out/'

In [ ]:
#!rm -r '../working/weights_out/'

# Training model on 97k of some 100k images of Coco_Train2017 dataset. Using the balance 3k as the Validation Set. Using all the 5k images of Coco_Val2017 dataset as the Test dataset.

## First phase of training already done and pickled various files to resume training with the data being split in a deterministic manner.

## Will not do all the steps again and simply load all the info from the pickled files:
## Descriptions, image_encodings, embedding_matrix, etc.

### The encoder is pre-trained Google Inception-v3 trained on Imagenet.

## AVAILABLE data :
### Coco_Train2017     = has 118287 images
### Coco_Val2017       = has 5000   images
### Combined total     = 123287     images

## USED data :
### Coco_Train2017     = 100000  images
### Coco_Val2017       = 5000    images
### Combined total     = 105000  images

### Using only the first 100k images of Train2017 + all 5k images of Val2017
### Thus total data available for training = 100k + 5k = 105k
### Details of split of data:
### Training   data = 97000 images from Coco_Train2017
### Validation data = 3000  images from Coco_Train2017
### Test       data = 5000  images from Coco_Val2017

### Note: 1) Each image will have multiple captions (up to 5 as some may be discarded)
###       2) Not using the Coco_Test2017 dataset at all as it has no annotations json file which has the captions.

## Using this LINK: https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8
## Image captioning with Keras

In [9]:
import os
import sys
import json
import time
import datetime
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import re
import pickle
#import itertools
import PIL
import PIL.Image

## import numpy as np
## from numpy import array
## import pandas as pd
## import matplotlib.pyplot as plt
## %matplotlib inline
## import string
## import os
## from PIL import Image
## import glob
## from pickle import dump, load
## from time import time
## from keras.preprocessing import sequence
## from keras.models import Sequential
## from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
## from keras.optimizers import Adam, RMSprop
## from keras.layers.wrappers import Bidirectional
## from keras.layers.merge import add
## from keras.applications.inception_v3 import InceptionV3
## from keras.preprocessing import image
## from keras.models import Model
## from keras import Input, layers
## from keras import optimizers
## from keras.applications.inception_v3 import preprocess_input
## from keras.preprocessing.text import Tokenizer
## from keras.preprocessing.sequence import pad_sequences
## from keras.utils import to_categorical

In [ ]:
#from google.colab import drive
#drive.flush_and_unmount()
#drive.mount('/content/gdrive')

In [10]:
## Kaggle versions

## Weights from training till now
OPDIR = r'../working/'

## New weights to save here
OPDIR_WEIGHTS = r'../working/weights_out/'

## Images locations
IPDIR_IMGS_COCO_TRAIN = r'../input/coco-2017-dataset/coco2017/train2017/'
IPDIR_IMGS_COCO_VAL = r'../input/coco-2017-dataset/coco2017/val2017/'

## to make deterministic - saved all the variables based on the data used to train during phase 1
## ALL SUBSEQUENT PHASES WILL RELOAD FROM THE DATA IN THIS LOCATION
IPDIR_DETERMINISTIC = '../input/thesis-imgcap-run2-deterministic-info/'

## Weights from previous stage of training
IPDIR_WEIGHTS_IN = r'../input/imgcap-kagg-run2-weights-in/'


#WORD_EMBEDDINGS_DIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/WordEmbeddings/' #not needed as already created the embedding matrix and pickled

## Google drive versions
#OPDIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/ImgCapTraining/CocoVal2017_1/'
#IPDIRIMGS = '/content/gdrive/My Drive/ThesisStoryGen/Data/coco_val2017_wget_1/val2017/'
#IPDIRANNO = '/content/gdrive/My Drive/ThesisStoryGen/Data/coco_annotations_trainval2017/'
#WORD_EMBEDDINGS_DIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/WordEmbeddings/'

## Data Preparation using Generator Function

In [ ]:
## Hereafter, I will try to explain the remaining steps by taking a sample example as follows:
## Consider we have 3 images and their 3 corresponding captions as follows:
## 
## (Train image 1) Caption -> The black cat sat on grass
## (Train image 2) Caption -> The white cat is walking on road
## (Test image) Caption -> The black cat is walking on grass
## 
## Now, let’s say we use the first two images and their captions to train the model and the third image to test our model.
## Now the questions that will be answered are: how do we frame this as a supervised learning problem?, what does the data matrix look like? how many data points do we have?, etc.
## First we need to convert both the images to their corresponding 2048 length feature vector as discussed above. Let “Image_1” and “Image_2” be the feature vectors of the first two images respectively
## Secondly, let’s build the vocabulary for the first two (train) captions by adding the two tokens “startseq” and “endseq” in both of them: (Assume we have already performed the basic cleaning steps)
## 
## Caption_1 -> “startseq the black cat sat on grass endseq”
## Caption_2 -> “startseq the white cat is walking on road endseq”
## 
## vocab = {black, cat, endseq, grass, is, on, road, sat, startseq, the, walking, white}
## 
## Let’s give an index to each word in the vocabulary:
## black -1, cat -2, endseq -3, grass -4, is -5, on -6, road -7, sat -8, startseq -9, the -10, walking -11, white -12
## 
## Now let’s try to frame it as a supervised learning problem where we have a set of data points D = {Xi, Yi}, where Xi is the feature vector of data point ‘i’ and Yi is the corresponding target variable.
## 
## Let’s take the first image vector Image_1 and its corresponding caption “startseq the black cat sat on grass endseq”. Recall that, Image vector is the input and the caption is what we need to predict. But the way we predict the caption is as follows:
## For the first time, we provide the image vector and the first word as input and try to predict the second word, i.e.:
## Input = Image_1 + ‘startseq’; Output = ‘the’
## Then we provide image vector and the first two words as input and try to predict the third word, i.e.:
## Input = Image_1 + ‘startseq the’; Output = ‘cat’
## And so on . . .
## 
## Thus, we can summarize the data matrix for one image and its corresponding caption as follows:
## Step 1 :: image feature = FeatVecImg1 :: Partial Caption = "startseq" :: Target Word = "the"
## Step 2 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the" :: Target Word = "black"
## Step 3 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black" :: Target Word = "cat"
## Step 4 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black cat" :: Target Word = "sat"
## Step 5 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black cat sat" :: Target Word = "on"
## Step 6 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black cat sat on" :: Target Word = "grass"
## Step 7 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black cat sat on grass" :: Target Word = "endseq"
## 
## It must be noted that, one image+caption is not a single data point but are multiple data points depending on the length of the caption.
## All the above 7 data points together constitute the full data for one image and its caption!!!!
## Similarly for second images, there will be multiple steps together that consitute its full data point.
## 
## We must now understand that in every data point, it’s not just the image which goes as input to the system, but also, a partial caption which helps to predict the next word in the sequence.
## Since we are processing sequences, we will employ a Recurrent Neural Network to read these partial captions (more on this later).
## However, we have already discussed that we are not going to pass the actual English text of the caption, rather we are going to pass the sequence of indices where each index represents a unique word.
## 
## Since we have already created an index for each word, let’s now replace the words with their indices and understand how the data matrix will look like:
## Step 1  :: image feature = FeatVecImg1 :: Partial Caption = "9" :: Target Word = "10"
## Step 2  :: image feature = FeatVecImg1 :: Partial Caption = "9 10" :: Target Word = "1"
## Step 3  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1" :: Target Word = "2"
## Step 4  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1 2" :: Target Word = "8"
## Step 5  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1 2 8" :: Target Word = "6"
## Step 6  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1 2 8 6" :: Target Word = "4"
## Step 7  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1 2 8 6 4" :: Target Word = "3"
## Step 8  :: image feature = FeatVecImg2 :: Partial Caption = "9" :: Target Word = "10"
## Step 9  :: image feature = FeatVecImg2 :: Partial Caption = "9 10" :: Target Word = "12"
## Step 10 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12" :: Target Word = "2"
## Step 11 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2" :: Target Word = "5"
## Step 12 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2 5" :: Target Word = "11"
## Step 13 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2 5 11" :: Target Word = "6"
## Step 14 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2 5 11 6" :: Target Word = "7"
## Step 15 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2 5 11 6 3" :: Target Word = "3"
## 
## Since we would be doing batch processing (explained later), we need to make sure that each sequence is of equal length. Hence we need to append 0’s (zero padding) at the end of each sequence. But how many zeros should we append in each sequence?
## Well, this is the reason we had calculated the maximum length of a caption. So we will append those many number of zeros which will lead to every sequence having a length = maximum length of caption.
## 
## Step 1  :: image feature = FeatVecImg1 :: Partial Caption = "[9 0 0 ...]" :: Target Word = "10"
## Step 2  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 0 0 ...]" :: Target Word = "1"
## Step 3  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 0 0 ...]" :: Target Word = "2"
## Step 4  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 2 0 0 ...]" :: Target Word = "8"
## Step 5  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 2 8 0 0 ...]" :: Target Word = "6"
## Step 6  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 2 8 6 0 0 ...]" :: Target Word = "4"
## Step 7  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 2 8 6 4 0 0 ...]" :: Target Word = "3"
## Step 8  :: image feature = FeatVecImg2 :: Partial Caption = "[9 0 0 ...]" :: Target Word = "10"
## Step 9  :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 0 0 ...]" :: Target Word = "12"
## Step 10 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 0 0 ...]" :: Target Word = "2"
## Step 11 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 0 0 ...]" :: Target Word = "5"
## Step 12 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 5 0 0 ...]" :: Target Word = "11"
## Step 13 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 5 11 0 0 ...]" :: Target Word = "6"
## Step 14 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 5 11 6 0 0 ...]" :: Target Word = "7"
## Step 15 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 5 11 6 3 0 0 ...]" :: Target Word = "3"
## Appended adequete 0's to each partial caption to make its length = max length
## 
## 
## 
## Need for a Data Generator:
## In the above example, I have only considered 2 images and captions which have lead to 15 data points.
## However, in our actual training dataset we have 6000 images, each having 5 captions. This makes a total of 30000 images and captions.
## Even if we assume that each caption on an average is just 7 words long, it will lead to a total of 30000*7 i.e. 210000 data points.
## 
## Compute the size of the data matrix:
## 
## 
##    img_vector = 2048        partial caption (length = max caption length)
## ---------------------------------------------------------------------------
## -                       -                                                 -
## -                       -                                                 -
## -                       -                                                 -
## -                       -                                                 -
## ---------------------------------------------------------------------------
## Say above matrix has n rows, m columns, so Size of the data matrix = n*m
## n-> number of data points (assumed as 210000)  (6000 images * 5 captions per image * 7 words average legnth of each caption)
## m-> length of each data point
##     = 2048 + length of partial caption (say something)
##     = 2048 + something
## 
## Now this "something"  NOTE EQUAL to the max length of caption!
## Every word (or index) will be mapped (embedded) to higher dimensional space through one of the word embedding techniques.
## As we using GloVe-200, each embedding has 200 floats representing each number.
## 
## So with each caption sentence consisting of max length caption size word-indexes, each word represented by 200 dimensional value:
##    Assuming max lenght of caption = 45
##    means "something" = 2048 + ( 45 * 200 ) = 2048 + 9000 = 11048 float values to represent each sentence
## 
## Therefore, size of data matrix = m*n = 210000 * 11048 = xxx float values!!!
## Assuming float takes 2 bytes (very conservative), that still means xxx * 2 = xxx GB
## 
## This is pretty huge requirement and even if we are able to manage to load this much data into the RAM, it will make the system very slow.
## For this reason we use data generators a lot in Deep Learning. Data Generators are a functionality which is natively implemented in Python. The ImageDataGenerator class provided by the Keras API is nothing but an implementation of generator function in Python.
## 
## So how does using a generator function solve this problem?
## If you know the basics of Deep Learning, then you must know that to train a model on a particular dataset, we use some version of Stochastic Gradient Descent (SGD) like Adam, Rmsprop, Adagrad, etc.
## With SGD, we do not calculate the loss on the entire data set to update the gradients. Rather in every iteration, we calculate the loss on a batch of data points (typically 64, 128, 256, etc.) to update the gradients.
## 
## This means that we do not require to store the entire dataset in the memory at once. Even if we have the current batch of points in the memory, it is sufficient for our purpose.
## A generator function in Python is used exactly for this purpose. It’s like an iterator which resumes the functionality from the point it left the last time it was called.
## To understand more about Generators, please read here (https://wiki.python.org/moin/Generators).
## 

In [11]:
# data generator, use during the call to model.fit_generator() to create batchwise data
def data_generator_1(_descriptions, _imgs_features_arr, _wordtoix, _max_length, _images_per_batch, _vocab_size):
    X1, X2, y = [] , [] , []  ## empty lists to populate the input and target data for a bath
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in _descriptions.items():
            n+=1
            # retrieve the encoded features of image
            img_feat = _imgs_features_arr[ key ] ## keys in the image encodings dict and descriptions dict use image_filename without the .jpg
            for desc in desc_list:
                # encode the sequence
                seq = [_wordtoix[word] for word in desc.split(' ') if word in _wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = keras.preprocessing.sequence.pad_sequences([in_seq], maxlen=_max_length)[0]
                    # encode output sequence
                    out_seq = keras.utils.to_categorical([out_seq], num_classes=_vocab_size)[0]
                    # store
                    X1.append(img_feat)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n == _images_per_batch:
                ## ValueError: No gradients provided for any variable: ['dense_2/kernel:0', 'dense_2/bias:0', 'lstm_1/lstm_cell_1/kernel:0', 'lstm_1/lstm_cell_1/recurrent_kernel:0', 'lstm_1/lstm_cell_1/bias:0', 'dense_3/kernel:0', 'dense_3/bias:0', 'dense_4/kernel:0', 'dense_4/bias:0'].
                #yield [[np.array(X1), np.array(X2)], np.array(y)]
                yield [np.array(X1), np.array(X2)], np.array(y)
                X1, X2, y = list(), list(), list()
                n=0

# Reload everything needed from pickled files created based on a certain split of Train and Validation datasets 97k:3k.
## Run necessary code again to check all good to proceed with Training phase:
## Outcome expected:
## 1) unique words in original vocabulary = 24323
## 2) unique words in culled vocab vocab_threshold = 6757
##    Thus, VOCAB_SIZE = 6757 + 1 = 6758
## 3) Embedding matrix shape should be (6758,200)

In [12]:
IPDIR_DETERMINISTIC

'../input/thesis-imgcap-run2-deterministic-info/'

In [13]:
os.listdir(IPDIR_DETERMINISTIC)

['ixtoword_train_97000.pkl',
 'descriptions_test.pkl',
 'img_encodings_val_3000.pkl',
 'img_encodings_train_97000.pkl',
 'descriptions_train_97000.pkl',
 'wordtoix_train_97000.pkl',
 'embedding_matrix_6758_200.pkl',
 'descriptions_train_97000_startend.pkl',
 'descriptions_val_3000.pkl',
 'descriptions_train_and_val.pkl',
 'val_imgs_keys_3000_randomSplit444.pkl',
 'train_imgs_keys_97000_randomSplit444.pkl']

In [14]:
## Reload all the files to make it deterministic


# variable = img_encodings_train
with open(IPDIR_DETERMINISTIC + 'img_encodings_train_97000.pkl', 'rb') as handle:
    img_encodings_train = pickle.load(handle)

# variable = img_encodings_val
with open(IPDIR_DETERMINISTIC + 'img_encodings_val_3000.pkl', 'rb') as handle:
    img_encodings_val = pickle.load(handle)

# variable = descriptions_train
with open(IPDIR_DETERMINISTIC + 'descriptions_train_97000_startend.pkl', 'rb') as handle:
    descriptions_train = pickle.load(handle)

# variable = descriptions_val
with open(IPDIR_DETERMINISTIC + 'descriptions_val_3000.pkl', 'rb') as handle:
    descriptions_val = pickle.load(handle)

# variable = train_imgs_keys
with open(IPDIR_DETERMINISTIC + 'train_imgs_keys_97000_randomSplit444.pkl', 'rb') as handle:
    train_imgs_keys = pickle.load(handle)

# variable = val_imgs_keys
with open(IPDIR_DETERMINISTIC + 'val_imgs_keys_3000_randomSplit444.pkl', 'rb') as handle:
    val_imgs_keys = pickle.load(handle)

# variable = embedding_matrix
with open(IPDIR_DETERMINISTIC + 'embedding_matrix_6758_200.pkl', 'rb') as handle:
    embedding_matrix = pickle.load(handle)

# variable = ixtoword
with open(IPDIR_DETERMINISTIC + 'ixtoword_train_97000.pkl', 'rb') as handle:
    ixtoword = pickle.load(handle)

# variable = wordtoix
with open(IPDIR_DETERMINISTIC + 'wordtoix_train_97000.pkl', 'rb') as handle:
    wordtoix = pickle.load(handle)

print(f"Reloaded it all")

Reloaded it all


In [ ]:
## check the reloaded files

In [15]:
print(f"Encodings data:\nlen(img_encodings_train) = {len(img_encodings_train)}\t\tlen(img_encodings_val) = {len(img_encodings_val)}")
print(f"Descriptions data:\nlen(descriptions_train) = {len(descriptions_train)}\t\tlen(descriptions_val) = {len(descriptions_val)}")
print(f"\nCHECK : reloaded values = 97k for Train , 3k for Validation")

Encodings data:
len(img_encodings_train) = 97000		len(img_encodings_val) = 3000
Descriptions data:
len(descriptions_train) = 97000		len(descriptions_val) = 3000

CHECK : reloaded values = 97k for Train , 3k for Validation


In [16]:
## at this stage the descriptions_train already has the start and end tokens added to it
vocabulary = set()
for key in descriptions_train.keys():
    [vocabulary.update(d.split()) for d in descriptions_train[key]]
print(f"Original Vocabulary Size with all words = {len(vocabulary)}")
print(f"\nCHECK : reloaded value = 24323")

Original Vocabulary Size with all words = 24323

CHECK : reloaded value = 24323


In [17]:
# Create a list of all the training captions, find the freq and retain words where the freq > threshold chosen

all_desc_in_training_samples = []
for key, val in descriptions_train.items():
    for cap in val:
        all_desc_in_training_samples.append(cap)

MIN_WORD_COUNT_THRESHOLD = 10
word_counts = {}
nsents = 0
for each_desc in all_desc_in_training_samples:
    nsents += 1
    for w in each_desc.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab_threshold = [w for w in word_counts if word_counts[w] >= MIN_WORD_COUNT_THRESHOLD]

print(f"Culled vocabulary to only retain words occurring more than threshold = {MIN_WORD_COUNT_THRESHOLD} times.\nNew vocab size , len(vocab_threshold) = {len(vocab_threshold)}")
print(f"\nCHECK : reloaded value = 6757")

Culled vocabulary to only retain words occurring more than threshold = 10 times.
New vocab size , len(vocab_threshold) = 6757

CHECK : reloaded value = 6757


In [18]:
## determine the maximum sequence length - parameter MAX_LENGTH_CAPTION used during the RNN deocder model setup

## convert a dictionary of clean descriptions to a list of descriptions
def extract_each_desc(_descriptions):
    all_desc = list()
    for key in _descriptions.keys():
        [all_desc.append(d) for d in _descriptions[key]]
    return all_desc

## find the longest description length
def find_max_length_desc(_descriptions):
    desc_sentences = extract_each_desc(_descriptions)
    return max(len(d.split()) for d in desc_sentences)

MAX_LENGTH_CAPTION = find_max_length_desc(descriptions_train)  ## will be used directly later while defining Decoder model
print(f"Max Description Length: {MAX_LENGTH_CAPTION}")
print(f"\nCHECK : reloaded value = 49")

Max Description Length: 49

CHECK : reloaded value = 49


In [19]:
## the value now, as it will be used as:: VOCAB_SIZE = len(wordtoix) + 1
print(f"len(wordtoix) = {len(wordtoix)}")
print(f"\nCHECK : reloaded value = 6757")

VOCAB_SIZE = len(wordtoix) + 1
print(f"\n\nSet the   VOCAB_SIZE = len(wordtoix) + 1 = {VOCAB_SIZE}")

EMBEDDING_DIMS = 200
print(f"\n\nSet the   EMBEDDING_DIMS = {EMBEDDING_DIMS}")

len(wordtoix) = 6757

CHECK : reloaded value = 6757


Set the   VOCAB_SIZE = len(wordtoix) + 1 = 6758


Set the   EMBEDDING_DIMS = 200


In [20]:
## see the index output by wordtoix for the start and end sequence tokens as well as some random one word
print( wordtoix.get('startseq') , wordtoix.get('endseq') , wordtoix.get('cat') )

1 9 526


In [21]:
print(f"Shape of embedding_matrix = {embedding_matrix.shape}")
print(f"\nCHECK : reloaded shape = 6758,200")

Shape of embedding_matrix = (6758, 200)

CHECK : reloaded shape = 6758,200


In [22]:
## Recheck these parameters that will define the Decoder architecture ::: EMBEDDING_DIMS , VOCAB_SIZE , MAX_LENGTH_CAPTION
print(f"SHOULD be:\nEMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49")
print(f"\nThe values to be used in Decoder setup:\nEMBEDDING_DIMS = {EMBEDDING_DIMS} , VOCAB_SIZE = {VOCAB_SIZE} , MAX_LENGTH_CAPTION = {MAX_LENGTH_CAPTION}")

SHOULD be:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49

The values to be used in Decoder setup:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49


## ALWAYS - Define the RNN Decoder model
## Reload ALL weights from previous training point. Then freeze ONLY the embedding layer as before. Thus, the embedding layer will continue to use the GloVe-200 embeddings matrix weights that were first set during Training Phase 1 setup.


### About Keras Emebedding layer
### 
#### Default initializing of this layer: https://keras.io/api/layers/core_layers/embedding/
######tf.keras.layers.Embedding(
######    input_dim, output_dim,
######    embeddings_initializer="uniform",
######    embeddings_regularizer=None,
######    activity_regularizer=None,
######    embeddings_constraint=None,
######    mask_zero=False,
######    input_length=None,
######    **kwargs
######)

In [23]:
IPDIR_WEIGHTS_IN

'../input/imgcap-kagg-run2-weights-in/'

In [24]:
os.listdir(IPDIR_WEIGHTS_IN)

['Decoder_Run_2_Wt_ep_7.h5',
 'Decoder_Run_2_Wt_ep_5.h5',
 'Decoder_Run_2_Wt_ep_1.h5',
 'Decoder_Run_2_Wt_ep_4.h5',
 'Decoder_Run_2_Wt_ep_2.h5',
 'Decoder_Run_2_Wt_ep_3.h5',
 'Decoder_Run_2_Wt_ep_6.h5']

## IMPORTANT - SPECIFY CORRECT WEIGHTS FILE TO LOAD FROM

In [25]:
RELOAD_WEIGHTS_FILE_NAME = r'Decoder_Run_2_Wt_ep_7.h5'

In [26]:
RELOAD_WEIGHTS_FILE_PATH = IPDIR_WEIGHTS_IN + RELOAD_WEIGHTS_FILE_NAME
print(f"Will reload this file = {RELOAD_WEIGHTS_FILE_PATH}")
print(f"Verfiy the weights in file exists : Check value is True = {os.path.exists(RELOAD_WEIGHTS_FILE_PATH) and os.path.isfile(RELOAD_WEIGHTS_FILE_PATH)}")

Will reload this file = ../input/imgcap-kagg-run2-weights-in/Decoder_Run_2_Wt_ep_7.h5
Verfiy the weights in file exists : Check value is True = True


In [27]:
def get_model_parameter_counts(_model):
    total_params = _model.count_params()
    total_trainable_params = int(np.sum([np.prod(v.get_shape().as_list()) for v in _model.trainable_variables]))
    total_non_trainable_params = int(np.sum([np.prod(v.get_shape().as_list()) for v in _model.non_trainable_variables]))
    #print(f"Total params = {total_decoder_params}\t Trainable total = {total_decoder_trainable_params}\t Non-trainable total = {total_decoder_non_trainable_params}")
    return total_params, total_trainable_params, total_non_trainable_params

def reload_rnn_encoder_for_more_training(_saved_weights_file, _EMBEDDING_DIMS, _VOCAB_SIZE, _MAX_LENGTH_CAPTION):
    if os.path.exists(_saved_weights_file) and os.path.isfile(_saved_weights_file):
        ## Decoder Model defining
        
        ## parameters to define model - sent during function call
        #EMBEDDING_DIMS is initialised earlier while creating embedding matrix
        #VOCAB_SIZE is initialised earlier
        #MAX_LENGTH_CAPTION is initialised earlier
        
        inputs1 = keras.Input(shape=(2048,))
        fe1 = keras.layers.Dropout(0.5)(inputs1)
        fe2 = keras.layers.Dense(256, activation='relu')(fe1)
        
        # partial caption sequence model
        inputs2 = keras.Input(shape=(_MAX_LENGTH_CAPTION,))
        se1 = keras.layers.Embedding(_VOCAB_SIZE, _EMBEDDING_DIMS, mask_zero=True)(inputs2)
        se2 = keras.layers.Dropout(0.5)(se1)
        se3 = keras.layers.LSTM(256)(se2)
        
        # decoder (feed forward) model
        decoder1 = keras.layers.add([fe2, se3])
        decoder2 = keras.layers.Dense(256, activation='relu')(decoder1)
        outputs = keras.layers.Dense(_VOCAB_SIZE, activation='softmax')(decoder2)
        
        # merge the two input models
        reloaded_rnn_decoder_model = keras.models.Model(inputs=[inputs1, inputs2], outputs=outputs)
        
        total_p, trainable_p, non_trainable_p = get_model_parameter_counts(reloaded_rnn_decoder_model)
        print(f"\nCreated RNN Decoder model defined with these paramenters:\nEMBEDDING_DIMS = {_EMBEDDING_DIMS} , VOCAB_SIZE = {_VOCAB_SIZE} , MAX_LENGTH_CAPTION = {_MAX_LENGTH_CAPTION}")
        print(f"\nBEFORE LOADING WEIGHTS AND FREEZING, parameter counts:\nTotal parameters = {total_p} , Trainable parameter = {trainable_p} , Non-trainable parameters = {non_trainable_p}")
        
        
        print(f"\nAttempting to load weights...\n")
        
        ## load the weights
        reloaded_rnn_decoder_model.load_weights(_saved_weights_file)
        print(f"SUCCESS - Reloaded weights from :: {_saved_weights_file}")
        
        ## freeze the embeddings layer weights so they are non-trainable
        reloaded_rnn_decoder_model.layers[2].trainable = False
        print(f"\nFrozen embeddings layer.")
        
        total_p, trainable_p, non_trainable_p = get_model_parameter_counts(reloaded_rnn_decoder_model)
        print(f"\nAFTER FREEZING, parameter counts:\nTotal parameters = {total_p} , Trainable parameter = {trainable_p} , Non-trainable parameters = {non_trainable_p}")
        
        return reloaded_rnn_decoder_model
    else:
        print(f"\nERROR reloading weights. Check weights file exists here = {_saved_weights_file} ;\nOR model setup parameters incompatible with the saved weights file given.")
        return None

In [28]:
print(f"The values to be used in Decoder setup:\nEMBEDDING_DIMS = {EMBEDDING_DIMS} , VOCAB_SIZE = {VOCAB_SIZE} , MAX_LENGTH_CAPTION = {MAX_LENGTH_CAPTION}")
print(f"Weights to be reloaded from here:\n{RELOAD_WEIGHTS_FILE_PATH}")

The values to be used in Decoder setup:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49
Weights to be reloaded from here:
../input/imgcap-kagg-run2-weights-in/Decoder_Run_2_Wt_ep_7.h5


In [29]:
reloaded_RNN_decoder = reload_rnn_encoder_for_more_training(RELOAD_WEIGHTS_FILE_PATH, EMBEDDING_DIMS, VOCAB_SIZE, MAX_LENGTH_CAPTION)
if reloaded_RNN_decoder is None:
    print(f"FATAL ERROR setting up Decoder")
else:
    print(f"\n{type(reloaded_RNN_decoder)}")


Created RNN Decoder model defined with these paramenters:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49

BEFORE LOADING WEIGHTS AND FREEZING, parameter counts:
Total parameters = 4146710 , Trainable parameter = 4146710 , Non-trainable parameters = 0

Attempting to load weights...

SUCCESS - Reloaded weights from :: ../input/imgcap-kagg-run2-weights-in/Decoder_Run_2_Wt_ep_7.h5

Frozen embeddings layer.

AFTER FREEZING, parameter counts:
Total parameters = 4146710 , Trainable parameter = 2795110 , Non-trainable parameters = 1351600

<class 'tensorflow.python.keras.engine.functional.Functional'>


In [30]:
## Output of command:  model_RNN_decoder.summary() AFTER FREEZING THE EMBEDDINGS WEIGHTS
## NOTE the count of Trainable params has REDUCED
reloaded_RNN_decoder.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 49)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 49, 200)      1351600     input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
_______________________________________________________________________________________

In [ ]:
## Output of command:  model_RNN_decoder.summary() AFTER FREEZING THE EMBEDDINGS WEIGHTS
## NOTE the count of Trainable params has REDUCED

#Model: "functional_3"
#__________________________________________________________________________________________________
#Layer (type)                    Output Shape         Param #     Connected to                     
#==================================================================================================
#input_4 (InputLayer)            [(None, 49)]         0                                            
#__________________________________________________________________________________________________
#input_3 (InputLayer)            [(None, 2048)]       0                                            
#__________________________________________________________________________________________________
#embedding_1 (Embedding)         (None, 49, 200)      1351600     input_4[0][0]                    
#__________________________________________________________________________________________________
#dropout_2 (Dropout)             (None, 2048)         0           input_3[0][0]                    
#__________________________________________________________________________________________________
#dropout_3 (Dropout)             (None, 49, 200)      0           embedding_1[0][0]                
#__________________________________________________________________________________________________
#dense_3 (Dense)                 (None, 256)          524544      dropout_2[0][0]                  
#__________________________________________________________________________________________________
#lstm_1 (LSTM)                   (None, 256)          467968      dropout_3[0][0]                  
#__________________________________________________________________________________________________
#add_1 (Add)                     (None, 256)          0           dense_3[0][0]                    
#                                                                 lstm_1[0][0]                     
#__________________________________________________________________________________________________
#dense_4 (Dense)                 (None, 256)          65792       add_1[0][0]                      
#__________________________________________________________________________________________________
#dense_5 (Dense)                 (None, 6758)         1736806     dense_4[0][0]                    
#==================================================================================================
#Total params: 4,146,710
#Trainable params: 2,795,110
#Non-trainable params: 1,351,600
#__________________________________________________________________________________________________

### ALWAYS - Compile RNN model

In [31]:
## setup the optimizer and compile

## see the default LR used, set some dummy to check it works
optimizer_adam = tf.keras.optimizers.Adam()  ## if nothing specified the default LR = 0.001
reloaded_RNN_decoder.compile(loss='categorical_crossentropy', optimizer=optimizer_adam)
print(f"Default LR without explicity setting so far = {reloaded_RNN_decoder.optimizer.learning_rate.numpy()}")

optimizer_adam.learning_rate.assign(0.5)
print(f"After setting LR as 0.5 = {reloaded_RNN_decoder.optimizer.learning_rate.numpy()}")

Default LR without explicity setting so far = 0.0010000000474974513
After setting LR as 0.5 = 0.5


## ALWAYS - Train the RNN Decoder

In [32]:
OPDIR_WEIGHTS

'../working/weights_out/'

In [33]:
os.listdir( OPDIR_WEIGHTS )

[]

In [34]:
## values setup earlier - for reference

print(f"\tEncodings data:\nlen(img_encodings_train) = {len(img_encodings_train)}\nlen(img_encodings_val) = {len(img_encodings_val)}")
print(f"\tDescriptions data:\nlen(descriptions_train) = {len(descriptions_train)}\nlen(descriptions_val) = {len(descriptions_val)}")

print(f"\nNO Validation Set being used for now.\n")

print(f"len(wordtoix) = {len(wordtoix)}")
print(f"VOCAB_SIZE = {VOCAB_SIZE}")
print(f"EMBEDDING_DIMS = {EMBEDDING_DIMS}")
print(f"embedding_matrix Shape = {embedding_matrix.shape}")
print(f"MAX_LENGTH_CAPTION = {MAX_LENGTH_CAPTION}")

print(f"\nInitialed with this input weights file:\n{RELOAD_WEIGHTS_FILE_PATH}")

	Encodings data:
len(img_encodings_train) = 97000
len(img_encodings_val) = 3000
	Descriptions data:
len(descriptions_train) = 97000
len(descriptions_val) = 3000

NO Validation Set being used for now.

len(wordtoix) = 6757
VOCAB_SIZE = 6758
EMBEDDING_DIMS = 200
embedding_matrix Shape = (6758, 200)
MAX_LENGTH_CAPTION = 49

Initialed with this input weights file:
../input/imgcap-kagg-run2-weights-in/Decoder_Run_2_Wt_ep_7.h5


In [35]:
MODEL_RUN_NUMBER = 2 ## large training data - 97k train
MODEL_RUN_NUMBER

2

In [36]:
## Specify the last epoch number (index from 1, NOT 0) so that the output weights files will have the correct epoch number for tracking
## E.g. if earlier phase ran up to Ep 7, then specify below as 7
EARLIER_TRAINING_EPOCH_END = 7
EARLIER_TRAINING_EPOCH_END

7

In [ ]:
########### Phase 4 #########

print(f"\n\nMODEL {MODEL_RUN_NUMBER} :: Training Phase 4 started at :: {datetime.datetime.now().strftime('%H:%M:%S')}\n")

LR_4 = 0.0001
BATCH_SIZE_4 = 64   ## how many images per batch
N_EPOCHS_4 = 3
STEPS_PER_EPOCH_4 = len(descriptions_train) // BATCH_SIZE_4
print(f"\nPhase 4 parameters:")
print(f"STEPS_PER_EPOCH_4 = {STEPS_PER_EPOCH_4}")
print(f"BATCH_SIZE_4 = {BATCH_SIZE_4}")
print(f"N_EPOCHS_4 = {N_EPOCHS_4}")

optimizer_adam.learning_rate.assign(LR_4)

for i in range( EARLIER_TRAINING_EPOCH_END , EARLIER_TRAINING_EPOCH_END + N_EPOCHS_4 ):
    print(f"\nEpoch {i+1} started at {datetime.datetime.now().strftime('%H:%M:%S')}\nLR used = {reloaded_RNN_decoder.optimizer.learning_rate.numpy()}\n")
    generator_1 = data_generator_1(descriptions_train, img_encodings_train, wordtoix, MAX_LENGTH_CAPTION, BATCH_SIZE_4, VOCAB_SIZE)
    reloaded_RNN_decoder.fit_generator(generator_1, epochs=1, steps_per_epoch=STEPS_PER_EPOCH_4, verbose=1)
    
    reloaded_RNN_decoder.save_weights( OPDIR_WEIGHTS + 'Decoder_Run_' + str(MODEL_RUN_NUMBER) + '_Wt_ep_' + str(i+1) + '.h5' )



MODEL 2 :: Training Phase 4 started at :: 16:56:07


Phase 4 parameters:
STEPS_PER_EPOCH_4 = 1515
BATCH_SIZE_4 = 64
N_EPOCHS_4 = 3

Epoch 8 started at 16:56:07
LR used = 9.999999747378752e-05

1515/1515 [==============================] - 7329s 5s/step - loss: 3.0858

Epoch 9 started at 18:58:27
LR used = 9.999999747378752e-05

1515/1515 [==============================] - 7264s 5s/step - loss: 3.0619

Epoch 10 started at 20:59:36
LR used = 9.999999747378752e-05

1144/1515 [=====================>........] - ETA: 29:58 - loss: 3.0474

In [ ]:
let this fail - console output at end of the Phase 3 training run

MODEL 2 :: Training Phase 4 started at :: 16:56:07


Phase 4 parameters:
STEPS_PER_EPOCH_4 = 1515
BATCH_SIZE_4 = 64
N_EPOCHS_4 = 3

Epoch 8 started at 16:56:07
LR used = 9.999999747378752e-05

1515/1515 [==============================] - 7329s 5s/step - loss: 3.0858

Epoch 9 started at 18:58:27
LR used = 9.999999747378752e-05

1515/1515 [==============================] - 7264s 5s/step - loss: 3.0619

Epoch 10 started at 20:59:36
LR used = 9.999999747378752e-05

1143/1515 [=====================>........] - ETA: 30:03 - loss: 3.0474

# KEEP TRACK OF EARLIER PHASE TRAINING

In [ ]:
LET THIS CELL FAIL IF RUN ACCIDENTALLY - KEPT TO CAPTURE HOW THE PHASE 1 TRAINING WAS DONE AND THE OUTPUT AT THE END

##########################################################
##########################################################
##########################################################

########### Phase 1 #########

print(f"\n\nMODEL {MODEL_RUN_NUMBER} :: Training Phase 1 started at :: {datetime.datetime.now().strftime('%H:%M:%S')}\n")

LR_1 = 0.0005
BATCH_SIZE_1 = 128   ## how many images per batch
N_EPOCHS_1 = 2
STEPS_PER_EPOCH_1 = len(descriptions_train) // BATCH_SIZE_1
print(f"\nPhase 1 parameters:")
print(f"STEPS_PER_EPOCH_1 = {STEPS_PER_EPOCH_1}")
print(f"BATCH_SIZE_1 = {BATCH_SIZE_1}")
print(f"N_EPOCHS_1 = {N_EPOCHS_1}")

optimizer_adam.learning_rate.assign(LR_1)

for i in range(N_EPOCHS_1):
    print(f"\nEpoch {i+1} started at {datetime.datetime.now().strftime('%H:%M:%S')}\nLR used = {model_RNN_decoder.optimizer.learning_rate.numpy()} \n")
    generator_1 = data_generator_1(descriptions_train, img_encodings_train, wordtoix, MAX_LENGTH_CAPTION, BATCH_SIZE_1, VOCAB_SIZE)
    model_RNN_decoder.fit_generator(generator_1, epochs=1, steps_per_epoch=STEPS_PER_EPOCH_1, verbose=1)
    
    model_RNN_decoder.save_weights( OPDIR_WEIGHTS + 'Decoder_Run_' + str(MODEL_RUN_NUMBER) + '_Wt_ep_' + str(i+1) + '.h5' )

##########################################################

The console output for Phase 1 training at the end:

MODEL 2 :: Training Phase 1 started at :: 17:15:11


Phase 1 parameters:
STEPS_PER_EPOCH_1 = 757
BATCH_SIZE_1 = 128
N_EPOCHS_1 = 2

Epoch 1 started at 17:15:11
LR used = 0.0005000000237487257 

757/757 [==============================] - 7705s 10s/step - loss: 4.3657

Epoch 2 started at 19:23:55
LR used = 0.0005000000237487257 

757/757 [==============================] - 7793s 10s/step - loss: 3.4819

##########################################################
##########################################################
##########################################################

???????????????????????
phase 2 is missing **************
??????????????????????


##########################################################
##########################################################
##########################################################

MODEL 2 :: Training Phase 3 started at :: 08:28:14


Phase 3 parameters:
STEPS_PER_EPOCH_3 = 757
BATCH_SIZE_3 = 128
N_EPOCHS_3 = 3

Epoch 5 started at 08:28:14
LR used = 0.00019999999494757503

757/757 [==============================] - 7752s 10s/step - loss: 3.1868

Epoch 6 started at 10:37:46
LR used = 0.00019999999494757503

757/757 [==============================] - 7795s 10s/step - loss: 3.1432

Epoch 7 started at 12:47:52
LR used = 0.00019999999494757503

757/757 [==============================] - 7784s 10s/step - loss: 3.1107

In [ ]:
#model_reset_states
#lr optimizer
#tuner